#链接谷歌云盘和Colab
需两次输入验证字符串，每次输入完按回车

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131289 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

#链接colab到谷歌云盘
#选择自己要进入的路径

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse -o nonempty drive
import os
os.chdir("drive")
!ls

aclImdb_v1.tar.gz     residual_plot.png      Titanic
data_description.txt  Sales		     torchtest.ipynb
drive		      sample_submission.csv  train.csv
GuodongDrive	      Sense1.ipynb	     Untitled0.ipynb
House		      Sense2.ipynb（副本）   XGboost_practice1.ipynb
Houseprice.ipynb      Sense.ipynb	     XGboost_practice2.ipynb
huawei		      sensePY.ipynb	     剑指offer
labeledTrainData.tsv  submission.csv	     数组和链表.desktop
neg		      test.csv		     糖尿病数据.txt
practice	      testData.tsv


In [3]:
os.chdir("practice/ch10")
!ls

original_data.xls  test_neural_network_data.xls   water_heater.xls
template.ipynb	   train_neural_network_data.xls


#开始

In [5]:
import pandas as pd
import numpy as np
from keras.layers.core import Activation,Dense
from keras.models import Sequential

Using TensorFlow backend.


#主要工作：特征处理，神经网络预测

programmer_1-->简单的数据筛选，划分数据

programmer_2-->阈值寻优

programmer_3-->建立训练神经网络，并进行模型的检验

programmer_4-->根据几个特征推算出是否满足某项条件

event_num-->相邻时间的差分，比较是否大于阈值


In [14]:
#阈值
thresshold = pd.Timedelta("4 min")
data = pd.read_excel('water_heater.xls')
#dataframe 处理
data[u"发生时间"] = pd.to_datetime(data[u"发生时间"], format="%Y%m%d%H%M%S")
data = data[data[u"水流量"]>0]
data.head()

,发生时间,开关机状态,加热中,保温中,实际温度,热水量,水流量,加热剩余时间,当前设置温度
2,2014-10-19 07:01:56,关,关,关,30°C,0%,8,0分钟,50°C
56,2014-10-19 07:38:16,关,关,关,30°C,0%,8,0分钟,50°C
381,2014-10-19 09:46:38,关,关,关,29°C,0%,16,0分钟,50°C
382,2014-10-19 09:46:40,关,关,关,29°C,0%,13,0分钟,50°C
384,2014-10-19 09:47:15,关,关,关,29°C,0%,20,0分钟,50°C


In [16]:
d = data[u"发生时间"].diff() > thresshold#相邻时间做差分
d.head()

2      False
56      True
381     True
382    False
384    False
Name: 发生时间, dtype: bool

In [19]:
data[u"事件编号"] = d.cumsum() + 1  # 通过累积求和的方式为事件编
data.head()

,发生时间,开关机状态,加热中,保温中,实际温度,热水量,水流量,加热剩余时间,当前设置温度,事件编号
2,2014-10-19 07:01:56,关,关,关,30°C,0%,8,0分钟,50°C,1
56,2014-10-19 07:38:16,关,关,关,30°C,0%,8,0分钟,50°C,2
381,2014-10-19 09:46:38,关,关,关,29°C,0%,16,0分钟,50°C,3
382,2014-10-19 09:46:40,关,关,关,29°C,0%,13,0分钟,50°C,3
384,2014-10-19 09:47:15,关,关,关,29°C,0%,20,0分钟,50°C,3


#Pro2

In [21]:
data = pd.read_excel('water_heater.xls')
#使用之后四个点的平均斜率
n = 4
#专家阈值
threshold = pd.Timedelta(minutes = 5)
data[u"发生时间"] = pd.to_datetime(data[u"发生时间"], format="%Y%m%d%H%M%S")
data = data[data[u"水流量"] > 0]
data.head()

,发生时间,开关机状态,加热中,保温中,实际温度,热水量,水流量,加热剩余时间,当前设置温度
2,2014-10-19 07:01:56,关,关,关,30°C,0%,8,0分钟,50°C
56,2014-10-19 07:38:16,关,关,关,30°C,0%,8,0分钟,50°C
381,2014-10-19 09:46:38,关,关,关,29°C,0%,16,0分钟,50°C
382,2014-10-19 09:46:40,关,关,关,29°C,0%,13,0分钟,50°C
384,2014-10-19 09:47:15,关,关,关,29°C,0%,20,0分钟,50°C


In [23]:
# 定义阈值列
dt = [pd.Timedelta(minutes=i) for i in np.arange(1, 9, 0.25)]
h = pd.DataFrame(dt, columns=[u"阈值"])
print(h)

         阈值
0  00:01:00
1  00:01:15
2  00:01:30
3  00:01:45
4  00:02:00
5  00:02:15
6  00:02:30
7  00:02:45
8  00:03:00
9  00:03:15
10 00:03:30
11 00:03:45
12 00:04:00
13 00:04:15
14 00:04:30
15 00:04:45
16 00:05:00
17 00:05:15
18 00:05:30
19 00:05:45
20 00:06:00
21 00:06:15
22 00:06:30
23 00:06:45
24 00:07:00
25 00:07:15
26 00:07:30
27 00:07:45
28 00:08:00
29 00:08:15
30 00:08:30
31 00:08:45


In [25]:
#没有看懂 
def event_num(ts):
      d = data[u"发生时间"].diff() > ts
      # 返回事件数
      return d.sum() + 1

# 计算每个阈值对应的事件数
h[u"事件数"] = h[u"阈值"].apply(event_num)
# 计算每两个相邻点对应的斜率
h[u"斜率"] = h[u"事件数"].diff() / 0.25
# 采用后n个的斜率绝对值平均作为斜率指标
h[u"斜率指标"] = pd.Series.rolling(h[u"斜率"].abs(), n).mean()
ts = h[u"阈值"][h[u"斜率指标"].idxmin() - n]

if ts > threshold:
    ts = pd.Timedelta(minutes=4)
print(ts)

0 days 00:04:00


In [26]:
threshold = pd.Timedelta("4 min")
inputfile = "water_heater.xls"
outputfile = "attribute_extract.xls"
data = pd.read_excel(inputfile)

data[u"发生时间"] = pd.to_datetime(data[u"发生时间"], format="%Y%m%d%H%M%S")
data = data[data[u"水流量"] > 0]
d = data[u"发生时间"].diff() > threshold
data[u"事件编号"] = d.cumsum() + 1

data_g = data.groupby(u"事件编号")
result = pd.DataFrame()
dt = pd.Timedelta(seconds=2)

for _, g in data_g:
    temp = pd.DataFrame(index=[0])
    # 根据用水时长、开关机切换次数、总用水量推出是否是洗澡
    tstart = g[u"发生时间"].min()
    tend = g[u"发生时间"].max()
    temp[u"用水事件时长（M）"] = (dt + tend - tstart).total_seconds() / 60
    temp[u"开关机切换次数"] = (pd.Series.rolling(g[u"开关机状态"] == u"关",
                                          2).sum() == 1).sum()
    temp[u"总用水量（L）"] = g[u"水流量"].sum()
    tdiff = g[u"发生时间"].diff()
    if len(g[u"发生时间"]) == 1:
        temp[u"总用水时长（Min）"] = dt.total_seconds() / 60
    else:
        temp[u"总用水时长（Min）"] = (
            tdiff.sum() - tdiff.iloc[1] / 2 -
            tdiff.iloc[len(tdiff) - 1] / 2).total_seconds() / 60
    temp[u"平均水流量（L/min）"] = temp[u"总用水量（L）"] / temp[u"总用水时长（Min）"]
    result = result.append(temp, ignore_index=True)
    
print(result)

     用水事件时长（M）  开关机切换次数  总用水量（L）  总用水时长（Min）  平均水流量（L/min）
0     0.033333        0        8    0.033333    240.000000
1     0.033333        0        8    0.033333    240.000000
2     0.650000        0       49    0.308333    158.918919
3     0.033333        0       22    0.033333    660.000000
4     0.033333        0        8    0.033333    240.000000
5    14.150000        1     7282   14.050000    518.291815
6     0.266667        0      384    0.183333   2094.545455
7    11.283333        0     3050    9.700000    314.432990
8     8.750000        0     3876    8.650000    448.092486
9     9.300000        0     4040    9.233333    437.545126
10    0.033333        0        8    0.033333    240.000000
11    0.166667        0       16    0.000000           inf
12    0.033333        0        8    0.033333    240.000000
13   15.933333        0     3895   13.216667    294.703657
14    0.066667        0       23    0.000000           inf
15    0.500000        0      496    0.433333   1144.6153